In [ ]:
# !pip install hmmlearn
# !pip install scikit-learn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm
import os
import glob
import pickle

In [3]:
def getFreqDictPerScore(directory_path):
    score_dict = {}
    for root, dirs, files in os.walk(directory_path):
        if not files:
            continue
        score = root.split("/")[1]
        freq_dict = {}
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as file:
                content = file.read()
                seq = content.split(" ")
                for s in seq:
                    if s not in freq_dict:
                        freq_dict[s] = 1
                    else:
                        freq_dict[s] += 1
        score_dict[score] = freq_dict
    return score_dict

score_dict = getFreqDictPerScore("essay_hmm")
for s, freq in score_dict.items():
    print(s, freq)

2.5 {'Leftclick': 6161, 'pause': 244101, 'Shift': 17923, 'q': 283674, 'Space': 71977, 'Backspace': 53864, '.': 3672, 'Enter': 2092, ',': 1438, 'NumLock': 14, 'ArrowLeft': 4354, 'End': 70, 'ArrowRight': 7525, 'ArrowDown': 2602, 'Delete': 552, "'": 388, '/': 35, ';': 121, 'b': 2, '=': 96, 'ArrowUp': 2300, '\\': 84, 'CapsLock': 1499, 'Rightclick': 40, '?': 84, '(': 10, ')': 14, 'Alt': 47, '"': 117, 'Control': 788, 'a': 7, 'c': 44, 'AudioVolumeMute': 6, '[': 33, '-': 108, 'Insert': 33, '#': 2, '~': 2, '`': 2, '>': 2, ':': 44, 'AltGraph': 1, 'Tab': 40, '<': 3, '!': 13, '%': 10, 'Home': 12, '$': 5, 'z': 55, 'F': 1, 'p': 1, 'v': 11, 'F11': 3, 'F1': 1, 'Clear': 4, '&': 3, 'I': 1, '*': 11, 'PageDown': 3, 'PageUp': 2, 'x': 5, 'Unidentified': 39, ']': 3, '0': 1, '1': 1, '2': 1, '\x80': 1, '@': 1, 'Meta': 14, 'F12': 2, '\x9b': 1, 's': 1, 'y': 7, 'Cancel': 1, 'Middleclick': 1, 'g': 4, 'A': 1, 'V': 1, '_': 1, '+': 3, 'n': 3}
6.0 {'Leftclick': 1325, 'pause': 33299, 'Shift': 5915, 'q': 142870, 'Space'

In [6]:
def getFreqDict(directory_path):
    freq_dict = {}
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as file:
                content = file.read()
                seq = content.split(" ")
                for s in seq:
                    if s not in freq_dict:
                        freq_dict[s] = 1
                    else:
                        freq_dict[s] += 1
    return freq_dict

freq_dict = getFreqDict("essay_hmm")
filtered_dict = {key: value for key, value in freq_dict.items() if value >= 1}
# for s in ['ArrowLeft', 'ArrowRight', 'ArrowDown']:
#     filtered_dict.pop(s)
filtered_dict

{'Leftclick': 91126,
 'pause': 2737371,
 'Shift': 334227,
 'q': 5365319,
 'Space': 1207128,
 'Backspace': 964089,
 '.': 59586,
 'Enter': 22689,
 ',': 46806,
 'NumLock': 273,
 'ArrowLeft': 63217,
 'End': 547,
 'ArrowRight': 121338,
 'ArrowDown': 30874,
 'Delete': 10965,
 "'": 11170,
 '/': 874,
 ';': 1785,
 'b': 7,
 '=': 1155,
 'ArrowUp': 28189,
 '\\': 554,
 'CapsLock': 12568,
 'Rightclick': 655,
 '?': 3155,
 '(': 611,
 ')': 578,
 'Alt': 466,
 '"': 4102,
 'Control': 4885,
 'a': 140,
 'c': 359,
 'AudioVolumeMute': 31,
 '[': 369,
 '-': 3843,
 'Insert': 226,
 '#': 17,
 '~': 4,
 '`': 18,
 '>': 89,
 ':': 634,
 'AltGraph': 4,
 'Tab': 1081,
 '<': 34,
 '!': 546,
 '%': 65,
 'Home': 221,
 '$': 97,
 'z': 207,
 'F': 1,
 'p': 5,
 'v': 311,
 'F11': 12,
 'F1': 1,
 'Clear': 7,
 '&': 60,
 'I': 1,
 '*': 69,
 'PageDown': 26,
 'PageUp': 18,
 'x': 127,
 'Unidentified': 6151,
 ']': 81,
 '0': 17,
 '1': 16,
 '2': 3,
 '\x80': 1,
 '@': 13,
 'Meta': 479,
 'F12': 15,
 '\x9b': 1,
 's': 44,
 'y': 9,
 'Cancel': 1,
 'M

In [ ]:
##reading from essay_hmm and generating a word dictionary for str -> int
def setWordDict(directory_path, word_dict):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as file:
                content = file.read()
                seq = content.split(" ")
                for s in seq:
                    if s not in word_dict and s in filtered_dict:
                        word_dict[s] = len(word_dict) 
    return word_dict
word_dict = {}
word_dict = setWordDict("essay_hmm", word_dict)
print(len(word_dict),word_dict)

In [ ]:
##convert the essay into int and get the max seq number
def convertStrToIntEssay(directory_path, word_dict):
    max_seq, max_file = 0, ""
    score_essay_list = []
    for root, dirs, files in os.walk(directory_path):
        if not files:
            continue
        score = root.split("/")[1]
        freq_dict = {}
        for file in files:
            file_path = os.path.join(root, file)
            essay_id = file.split(".")[0]
            with open(file_path, 'r') as file:
                content = file.read()
                seq = content.split(" ")
                essay_int = [word_dict[s] for s in seq if s in word_dict and s in filtered_dict]
                # max_seq = max(max_seq, len(essay_int))
                if len(essay_int) > max_seq:
                    max_seq, max_file = len(essay_int), file
            score_essay_list.append((score, essay_id, essay_int))
    return max_seq, max_file, score_essay_list

##max_13257
max_seq, max_file, score_essay_list = convertStrToIntEssay("essay_hmm", word_dict)
##list of tuple (essay_id, seq)

# print( len(essay_inputs[0][1]))


In [ ]:
print(score_essay_list[0])
print(max_seq,max_file,len(score_essay_list[0][2]))

In [ ]:
##keep duplicating itself until it reaches the max length
##essay_inputs is in [(essay_id, seq)] form
def padding_essay(essay_inputs, max_seq):
    for i, (score, file_name, seq) in enumerate(essay_inputs):
        while len(seq) < max_seq:
            seq.extend(seq)
        seq = seq[0:max_seq]
        essay_inputs[i] =(score, file_name, seq)

padding_essay(score_essay_list, max_seq)
all(len(tup[2]) == max_seq for tup in score_essay_list)

In [ ]:
def split_score(score_essay_list):
    # Group tuples by the first value
    grouped_dict = {}
    for item in score_essay_list:
        key = item[0]
        if key in grouped_dict:
            grouped_dict[key].append(item[2])
        else:
            grouped_dict[key] = [item[2]]

# Extract tuples with the same first value and put them into an array
    return grouped_dict

result_array = split_score(score_essay_list)
print(len(result_array['2.5']))

In [ ]:

list_of_sets = []
for s, hmm_input in result_array.items():
    one_d_array = [element for sublist in hmm_input for element in sublist]
    print(s, set(one_d_array))
    list_of_sets.append(set(one_d_array))
overlap_keys = set.intersection(*list_of_sets)
# print(overlap_keys)

In [ ]:
def findBestModel(n_fits, N, train_input, seed):
    best_score = best_model = None
    for idx in range(n_fits):
        model = hmm.CategoricalHMM(
        n_components=N, random_state=seed, 
        init_params='se')  # don't init transition, set it below
    # we need to initialize with random transition matrix probabilities
    # because the default is an even likelihood transition
    # we know transitions are rare (otherwise the casino would get caught!)
    # so let's have an Dirichlet random prior with an alpha value of
    # (0.1, 0.9) to enforce our assumption transitions happen roughly 10%
    # of the time

    ##A
        epsilon = 1e-2
        transmat = np.full((N, N), 1 / N) + epsilon
        transmat /= transmat.sum(axis=1, keepdims=True)
        random_noise = np.random.randn(N, N) * 1e-2  # Adjust the scale of the noise as needed
        transmat += random_noise
        transmat /= transmat.sum(axis=1, keepdims=True)
        # print(transmat)
    # model.transmat_ = np.array([np.random.dirichlet([0.9, 0.1]),
    #                             np.random.dirichlet([0.1, 0.9])])
        model.transmat_ = transmat
        model.fit(train_input)
        score = model.score(train_input)
        print(f'{seed} Model-{s} #{idx}\tScore: {score}')
        if best_score is None or score > best_score:
            best_model = model
            best_score = score
    return best_score, best_model

n_fits = 1
N = 1

for s, train_input in result_array.items():
    lowest_best_score, lowest_best_model = findBestModel(n_fits, N, train_input, N)
    if not os.path.exists("model/" + s):
                os.makedirs("model/" + s)
    with open("model/" + s + "/" + s + "-" + str(N) + "-model" + str(lowest_best_score) + ".pkl", "wb") as file:
            pickle.dump(lowest_best_model, file)

In [ ]:
score_essay_list[0]

In [ ]:
model_path = "model"
scores = [ "0.5", "1.0", "1.5", "2.0", "2.5", "3.0","3.5", "4.0","4.5", "5.0", "5.5",  "6.0"]
for score in scores:
    model_file = glob.glob(os.path.join(model_path, score, '*.pkl'))[0]
    with open(model_file, "rb") as file:
        model = pickle.load(file)
        print(model_file)
        print(model.transmat_)
        print(model.emissionprob_)

In [ ]:
X1 = [[0.5], [1.0], [-1.0], [0.42], [0.24]]
X2 = [[2.4], [4.2], [0.5], [-0.24]]
X = np.concatenate([X1, X2])
X

In [ ]:
import concurrent.futures
num_workers = 10
n_fits = 1
N = 5
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    for s, train_input, test_input in score_set:
        # arguments = [(n_fits, N, train_input, test_input)] * num_workers
        arguments = [(n_fits, N, train_input, test_input, i) for i in range(num_workers)]
        futures = [executor.submit(findBestModel, *arg) for arg in arguments]
        concurrent.futures.wait(futures)
        result = [future.result() for future in futures]
        print(result)
        best_result = min(result, key=lambda x: x[0])
        lowest_best_score, lowest_best_model = best_result
        if not os.path.exists("model/" + s):
                os.makedirs("model/" + s)
        with open("model/" + s + "/" + s + "-" + str(N) + "-model" + str(lowest_best_score) + ".pkl", "wb") as file:
            pickle.dump(lowest_best_model, file)

In [ ]:
def generateHMMScoreOnEssay(model_path, essay_inputs):
    ##read model
    model_list = []
    scores = [ "0.5", "1.0", "1.5", "2.0", "2.5", "3.0","3.5", "4.0","4.5", "5.0", "5.5",  "6.0"]
    result = []
    for score in scores:
        model_file = glob.glob(os.path.join(model_path, score, '*.pkl'))[0]
        with open(model_file, "rb") as file:
            model = pickle.load(file)
            model_list.append((score,model))
    for i, (essay_id, essay_input) in enumerate(essay_inputs):
        hmm_result = [essay_id]
        for s, model in model_list:
            print(f"{i}-Training essay {essay_id} against model-{s}")
            hmm_score = model.score([essay_input])
            hmm_result.append(hmm_score)
            # hmm_result.append(score)
        result.append(hmm_result)
    return scores, result

scores, hmm_result = generateHMMScoreOnEssay("model",essay_inputs[0:10])
hmm_result_df = pd.DataFrame(hmm_result)
col = ['id'] + scores
hmm_result_df.columns = col

In [ ]:
hmm_result_df.head(10)

In [ ]:
hmm_result_df.to_csv("essay_hmm_reult.csv")

In [ ]:
# ##reading from the text file for observation sequnce. Only using 4.0 for now

# def readInputFromFile(score):
#     file_path = score
#     with open(file_path, "r") as file:
#         lines = file.readlines()
#         seqs = lines[0].split("K")
#         words = " ".join(seqs).split(" ")
#         string_id = {word: i for i, word in enumerate(set(words))}

#         seqs = [s.split(" ") for s in seqs]
#         ##convert words in seqs to numbers
#         seqs = [[string_id[w] for w in s] for s in seqs]
#         # lines = file.readlines()
#         # print(len(lines))
#         # string_to_id = {string: i for i, string in enumerate(set(lines[0]))}
#         # raw_input = lines[0].split("K")
#         # print(len(raw_input))
#         # raw_string_pre_pad = [s.split(" ") for s in raw_input ]
#         #
#         # max_seq = max(len(s) for s in seqs)
#         # hmm_input = [np.pad(s, (0, max_seq - len(s)), constant_values=9999) for s in seqs]
#         hmm_input = seqs
#     return hmm_input

In [ ]:
# ##keep duplicating itself until it reaches the max length
# ##content is in "seqKseqK" string form
# def convertStrToIntSet(file_path, word_dict):
#     set_inputs = []
#     with open(file_path, 'r') as file:
#         content = file.read()
#         seqs = content.split("K")
#         # print(len(seqs))
#         for seq in seqs:
#             # print(seq)
#             seq_int = []
#             for s in seq.split(" "):
#                 if len(s) == 0 or s not in word_dict or s not in filtered_dict:
#                     continue
#                 seq_int.append(word_dict[s])
#             set_inputs.append(seq_int)
#     return set_inputs

# ##2d array that contains seq 
# score_set = []
# ##train input is a 2d array. First len is number of doc, second is sequence of doc
# # for s in ["1.0", "1.5", "2.0", "2.5", "3.0", "4.5", "5.0", "5.5","3.5", "4.0", "0.5", "6.0"]:
# for s in ["4.0"]:
#     train_seqs = convertStrToIntSet("train/" + s + ".txt", word_dict)
#     test_seqs = convertStrToIntSet("test/" + s + ".txt", word_dict)
#     # print(len(train_seqs))
#     score_set.append((s, train_seqs, test_seqs))

In [ ]:
# ##keep duplicating itself until it reaches the max length
# def padding_set(set_inputs, max_seq):
#     for i, seq in enumerate(set_inputs):
#         while len(seq) < max_seq:
#             seq.extend(seq)
#         seq = seq[0:max_seq]
#         set_inputs[i] = seq
# # def padding_set(set_inputs, max_seq):
# #     for i, seq in enumerate(set_inputs):
# #         seq = np.pad(seq, (0, max_seq - len(seq)), constant_values=len(word_dict))
# #         set_inputs[i] = seq[0:1000]
# ##calling padding
# for s, train_seqs, test_seqs in score_set:
#     padding_set(train_seqs, max_seq)
#     padding_set(test_seqs, max_seq)

# ##checking if all has same length
# for s, train_seqs, test_seqs in score_set:
#     print(s, len(train_seqs), len(test_seqs), all(len(seq) == max_seq for seq in train_seqs), all(len(seq) == max_seq for seq in test_seqs))
